In [2]:
from google.colab import files
uploaded = files.upload()

Saving test.data to test (1).data
Saving training.data to training (1).data


Uploading data files

In [4]:
import pandas as pd


column_names = [
    "A1", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9",
    "A10", "A11", "A12", "A13", "A14", "A15", "Class"
]

# Load the datasets
train_file = "training (1).data"
test_file = "test (1).data"

# Read CSV files
train_df = pd.read_csv(train_file, header=None, names=column_names, na_values='?')
test_df = pd.read_csv(test_file, header=None, names=column_names, na_values='?')

# Display first few rows
print(train_df.head())
print(test_df.head())


  A1     A2      A3 A4 A5  A6  A7     A8 A9 A10  A11 A12 A13    A14   A15  \
0  b  16.92   0.335  y  p   k   v  0.290  f   f    0   f   s  200.0     0   
1  b  48.50   4.250  u  g   m   v  0.125  t   f    0   t   g  225.0     0   
2  a  68.67  15.000  u  g   e   z  0.000  t   t   14   f   g    0.0  3376   
3  a  25.08   2.540  y  p  aa   v  0.250  t   f    0   t   g  370.0     0   
4  b  33.75   2.750  u  g   i  bb  0.000  f   f    0   f   g  180.0     0   

  Class  
0     -  
1     +  
2     +  
3     +  
4     -  
    A1     A2      A3 A4 A5  A6  A7     A8 A9 A10  A11 A12 A13    A14   A15  \
0    b  27.42  12.500  u  g  aa  bb  0.250  f   f    0   t   g  720.0     0   
1  NaN  28.17   0.585  u  g  aa   v  0.040  f   f    0   f   g  260.0  1004   
2    b  32.92   2.500  u  g  aa   v  1.750  f   t    2   t   g  720.0     0   
3    a  22.67   0.790  u  g   i   v  0.085  f   f    0   f   g  144.0     0   
4    b  22.67   0.750  u  g   i   v  1.585  f   t    1   t   g  400.0     9   

  

Handling missing values

In [5]:
import numpy as np


def handling_missing_values(train_df, test_df):
    for col in train_df.columns[:-1]:  # Exclude the last column(class)
        if train_df[col].isnull().sum() > 0:  # Checking missing values
            if train_df[col].dtype == 'O':  # Categorical attribute(String datatype)
                sorted_values = sorted(train_df[col].dropna().unique())  # Sort alphabetically
                median_index = len(sorted_values) // 2  # Get median index
                median_value = sorted_values[median_index]  # Pick median

            else:  # Continuous attribute (no sorting needed)
                median_value = train_df[col].median()

            # Replace missing values
            train_df[col].fillna(median_value, inplace=True)
            test_df[col].fillna(median_value, inplace=True)  # Use training set median for test set

    return train_df, test_df

# Handle missing values
train_df, test_df = handling_missing_values(train_df, test_df)

# Verify missing values are handled
print("Missing values in training set after handling:\n", train_df.isnull().sum())
print("Missing values in test set after handling:\n", test_df.isnull().sum())




Missing values in training set after handling:
 A1       0
A2       0
A3       0
A4       0
A5       0
A6       0
A7       0
A8       0
A9       0
A10      0
A11      0
A12      0
A13      0
A14      0
A15      0
Class    0
dtype: int64
Missing values in test set after handling:
 A1       0
A2       0
A3       0
A4       0
A5       0
A6       0
A7       0
A8       0
A9       0
A10      0
A11      0
A12      0
A13      0
A14      0
A15      0
Class    0
dtype: int64


<ipython-input-5-25b9ae91b7a9>:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(median_value, inplace=True)
<ipython-input-5-25b9ae91b7a9>:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

# **Implementing C4.5 Algorithm**

In [6]:
import numpy as np
import pandas as pd

class DecisionTreeC45:
    def __init__(self, data, target):
        self.data = data
        self.target = target
        self.tree = self.build_tree(self.data, self.target)

    def build_tree(self, data, target):
        """ Recursively builds the C4.5 Decision Tree """
        if len(set(target)) == 1:
            return {'class': target.iloc[0]}  # Pure node, return class

        if len(data.columns) == 0:
            return {'class': target.mode().iloc[0]}  # Majority class

        # Select the best attribute
        best_attribute, best_split = self.select_best_attribute(data, target)

        # Create the decision node
        decision_node = {'attribute': best_attribute, 'children': {}}

        if best_split is None:  # Categorical attribute
            for value in data[best_attribute].unique():
                subset_data = data[data[best_attribute] == value]
                subset_target = target[data[best_attribute] == value]
                decision_node['children'][value] = self.build_tree(subset_data.drop(columns=[best_attribute]), subset_target)
        else:  # Continuous attribute (bi-partitioning)
            decision_node['split'] = best_split
            left_data = data[data[best_attribute] <= best_split]
            right_data = data[data[best_attribute] > best_split]
            left_target = target[data[best_attribute] <= best_split]
            right_target = target[data[best_attribute] > best_split]

            decision_node['children']["≤ " + str(best_split)] = self.build_tree(left_data, left_target)
            decision_node['children']["> " + str(best_split)] = self.build_tree(right_data, right_target)

        return decision_node

    def select_best_attribute(self, data, target):
        """ Selects the best attribute based on Gain Ratio """
        gain_ratios = {}
        best_splits = {}

        for attribute in data.columns:
            if data[attribute].dtype == 'O':  # Categorical attribute
                gain_ratios[attribute] = self.calculate_gain_ratio(data[attribute], target)
                best_splits[attribute] = None
            else:  # Continuous attribute
                best_split, best_gain_ratio = self.find_best_split(data[attribute], target)
                gain_ratios[attribute] = best_gain_ratio
                best_splits[attribute] = best_split

        # Choose the attribute with the highest Gain Ratio
        best_attribute = max(gain_ratios, key=gain_ratios.get)
        return best_attribute, best_splits[best_attribute]

    def find_best_split(self, attribute_values, target):
        """ Finds the best split point for a continuous attribute. """
        sorted_values = attribute_values.sort_values().unique()
        candidate_splits = (sorted_values[:-1] + sorted_values[1:]) / 2  # Midpoints

        best_split = None
        best_gain_ratio = -1  # Start with a very low value

        for split in candidate_splits:
            left_target = target[attribute_values <= split]
            right_target = target[attribute_values > split]

            gain = self.calculate_information_gain_split(target, left_target, right_target)
            iv = self.calculate_intrinsic_value_split(left_target, right_target)

            gain_ratio = gain / iv if iv != 0 else 0  # Avoid division by zero

            if gain_ratio > best_gain_ratio:
                best_gain_ratio = gain_ratio
                best_split = split

        return best_split, best_gain_ratio

    def calculate_gain_ratio(self, attribute_values, target):
        """ Compute Gain Ratio for both categorical & continuous attributes """
        if attribute_values.dtype == 'O':  # Categorical
            info_gain = self.calculate_information_gain(attribute_values, target)
            intrinsic_value = self.calculate_intrinsic_value(attribute_values)
        else:  # Continuous
            best_split, info_gain = self.find_best_split(attribute_values, target)
            intrinsic_value = self.calculate_intrinsic_value_split(target[attribute_values <= best_split], target[attribute_values > best_split])

        return info_gain / intrinsic_value if intrinsic_value != 0 else 0  # Avoid division by zero

    def calculate_information_gain(self, attribute_values, target):
        """ Compute Information Gain for categorical attributes """
        entropy_total = self.calculate_entropy(target)

        weighted_entropy = sum(
            (len(subset_target) / len(target)) * self.calculate_entropy(subset_target)
            for value in attribute_values.unique()
            if (subset_target := target[attribute_values == value]).size > 0
        )

        return entropy_total - weighted_entropy

    def calculate_information_gain_split(self, full_target, left_target, right_target):
        """ Compute Information Gain for a continuous attribute split. """
        entropy_total = self.calculate_entropy(full_target)
        weighted_entropy = ((len(left_target) / len(full_target)) * self.calculate_entropy(left_target) +
                            (len(right_target) / len(full_target)) * self.calculate_entropy(right_target))
        return entropy_total - weighted_entropy

    def calculate_intrinsic_value(self, attribute_values):
        """ Compute Intrinsic Value for Gain Ratio (Categorical Attributes) """
        value_counts = attribute_values.value_counts()
        probabilities = value_counts / len(attribute_values)
        intrinsic_value = -sum(probabilities * np.log2(probabilities))
        return intrinsic_value

    def calculate_intrinsic_value_split(self, left_target, right_target):
        """ Compute Intrinsic Value for Gain Ratio (Continuous Attributes) """
        total_size = len(left_target) + len(right_target)
        p_left = len(left_target) / total_size
        p_right = len(right_target) / total_size
        iv = - (p_left * np.log2(p_left) if p_left > 0 else 0) - (p_right * np.log2(p_right) if p_right > 0 else 0)
        return iv

    def calculate_entropy(self, target):
        """ Compute entropy for a given target column. """
        class_counts = target.value_counts()
        probabilities = class_counts / len(target)
        return -sum(probabilities * np.log2(probabilities))

    def predict(self, examples):
        """ Predict class labels for given examples using the trained decision tree. """
        return examples.apply(lambda row: self._classify(row, self.tree), axis=1)

    def _classify(self, row, tree):
        """ Recursively classify a single example. """
        if 'class' in tree:
            return tree['class']

        attribute = tree['attribute']
        value = row.get(attribute, None)

        if "split" in tree:  # Continuous attribute
            split_value = tree["split"]
            if value is not None:
                branch = "≤ " + str(split_value) if value <= split_value else "> " + str(split_value)
            else:
                branch = "≤ " + str(split_value)  # Default to the left branch

        else:  # Categorical attribute
            branch = value if value in tree['children'] else self.target.mode().iloc[0]

        return self._classify(row, tree['children'].get(branch, tree['children'][list(tree['children'].keys())[0]]))


# **Implementing CART Algorithm**

In [7]:
import numpy as np
import pandas as pd

class DecisionTreeCART:
    def __init__(self, data, target):
        self.data = data
        self.target = target
        self.tree = self.build_tree(self.data, self.target)

    def build_tree(self, data, target):
        # Base Cases
        if len(set(target)) == 1:
            return {'class': target.iloc[0]}

        if len(data.columns) == 0:
            majority_class = target.mode().iloc[0]
            return {'class': majority_class}

        # Select the best attribute using Gini Index
        best_attribute, best_split = self.select_best_attribute(data, target)

        # Create a decision node
        decision_node = {'attribute': best_attribute, 'children': {}}

        if best_split is None:  # Categorical attribute (binary split)
            left_values, right_values = self.find_best_categorical_split(data[best_attribute], target)
            left_data = data[data[best_attribute].isin(left_values)]
            right_data = data[data[best_attribute].isin(right_values)]
            left_target = target[data[best_attribute].isin(left_values)]
            right_target = target[data[best_attribute].isin(right_values)]

            decision_node['children']['IN ' + str(left_values)] = self.build_tree(left_data, left_target)
            decision_node['children']['IN ' + str(right_values)] = self.build_tree(right_data, right_target)

        else:  # Continuous attribute (bi-partitioning)
            decision_node['split'] = best_split
            left_data = data[data[best_attribute] <= best_split]
            right_data = data[data[best_attribute] > best_split]
            left_target = target[data[best_attribute] <= best_split]
            right_target = target[data[best_attribute] > best_split]

            decision_node['children']["≤ " + str(best_split)] = self.build_tree(left_data, left_target)
            decision_node['children']["> " + str(best_split)] = self.build_tree(right_data, right_target)

        return decision_node

    def select_best_attribute(self, data, target):
        gini_scores = {}
        best_splits = {}

        for attribute in data.columns:
            if data[attribute].dtype == 'O':  # Categorical attribute
                best_gini, left_values, right_values = self.find_best_categorical_split(data[attribute], target, return_gini=True)
                gini_scores[attribute] = best_gini
                best_splits[attribute] = None  # No split value for categorical
            else:  # Continuous attribute
                best_split, best_gini = self.find_best_continuous_split(data[attribute], target)
                gini_scores[attribute] = best_gini
                best_splits[attribute] = best_split

        # Choose the attribute with the lowest Gini index
        best_attribute = min(gini_scores, key=gini_scores.get)
        return best_attribute, best_splits[best_attribute]

    def find_best_continuous_split(self, attribute_values, target):
        """ Finds the best split point for a continuous attribute based on Gini Index. """
        sorted_values = attribute_values.sort_values().unique()
        candidate_splits = (sorted_values[:-1] + sorted_values[1:]) / 2  # Midpoints

        best_split = None
        best_gini = float('inf')

        for split in candidate_splits:
            left_target = target[attribute_values <= split]
            right_target = target[attribute_values > split]

            gini = self.calculate_gini_split(left_target, right_target)

            if gini < best_gini:
                best_gini = gini
                best_split = split

        return best_split, best_gini

    def find_best_categorical_split(self, attribute_values, target, return_gini=False):
        """ Finds the best binary split for a categorical attribute. """
        unique_values = attribute_values.unique()
        best_gini = float('inf')
        best_split = (set(), set())

        for i in range(1, len(unique_values)):
            left_values = set(unique_values[:i])
            right_values = set(unique_values[i:])

            left_target = target[attribute_values.isin(left_values)]
            right_target = target[attribute_values.isin(right_values)]

            gini = self.calculate_gini_split(left_target, right_target)

            if gini < best_gini:
                best_gini = gini
                best_split = (left_values, right_values)

        if return_gini:
            return best_gini, best_split[0], best_split[1]
        return best_split

    def calculate_gini_split(self, left_target, right_target):
        """ Compute weighted Gini Index for a split. """
        total_size = len(left_target) + len(right_target)
        p_left = len(left_target) / total_size
        p_right = len(right_target) / total_size
        return p_left * self.calculate_gini(left_target) + p_right * self.calculate_gini(right_target)

    def calculate_gini(self, target):
        """ Compute Gini Index for a given target column. """
        class_counts = target.value_counts()
        probabilities = class_counts / len(target)
        return 1 - sum(probabilities ** 2)

    def predict(self, examples):
        """ Predict class labels for given examples using the trained decision tree. """
        return examples.apply(lambda row: self._classify(row, self.tree), axis=1)

    def _classify(self, row, tree):
        """ Recursively classify a single example. """
        if 'class' in tree:
            return tree['class']

        attribute = tree['attribute']
        value = row.get(attribute, None)

        if "split" in tree:  # Continuous attribute
            split_value = tree["split"]
            branch = "≤ " + str(split_value) if value <= split_value else "> " + str(split_value)
        else:  # Categorical attribute
            branch = "IN " + str(value) if value in tree['children'] else list(tree['children'].keys())[0]

        return self._classify(row, tree['children'].get(branch, tree['children'][list(tree['children'].keys())[0]]))


Implementing 10 fold cross validation

In [8]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

class CrossValidator:
    def __init__(self, data, target, model_type):
        self.data = data
        self.target = target
        self.model_type = model_type  # "C4.5" or "CART"
        self.folds = self.create_folds()

    def create_folds(self):
        """Splits data sequentially into 10 folds."""
        fold_size = len(self.data) // 10
        folds = []
        for i in range(10):
            start_idx = i * fold_size
            end_idx = start_idx + fold_size
            folds.append((self.data.iloc[start_idx:end_idx], self.target.iloc[start_idx:end_idx]))
        return folds

    def evaluate(self):
        """Performs 10-fold cross-validation and returns the average F1-score."""
        f1_scores = []

        for i in range(10):
            # Use fold i as validation, others as training
            val_data, val_target = self.folds[i]
            train_data = pd.concat([self.folds[j][0] for j in range(10) if j != i])
            train_target = pd.concat([self.folds[j][1] for j in range(10) if j != i])

            # Train model
            if self.model_type == "C4.5":
                model = DecisionTreeC45(train_data, train_target)
            else:  # "CART"
                model = DecisionTreeCART(train_data, train_target)

            # Predict and compute F1-score
            predictions = model.predict(val_data)
            f1 = f1_score(val_target, predictions, pos_label="+")  # Assuming "+" is the positive class
            f1_scores.append(f1)

        avg_f1 = np.mean(f1_scores)
        return avg_f1


Picking best model and reporting F1 scores

In [9]:
# Load training and test data
train_data = train_df.drop(columns=["Class"])
train_target = train_df["Class"]
test_data = test_df.drop(columns=["Class"])
test_target = test_df["Class"]

# Perform cross-validation for both models
cv_c45 = CrossValidator(train_data, train_target, "C4.5")
cv_cart = CrossValidator(train_data, train_target, "CART")

f1_c45 = cv_c45.evaluate()
f1_cart = cv_cart.evaluate()

# Select the best model
best_model_type = "C4.5" if f1_c45 > f1_cart else "CART"
print(f"Best Model: {best_model_type} (C4.5 F1: {f1_c45:.4f}, CART F1: {f1_cart:.4f})")

# Train the best model on the full training set
if best_model_type == "C4.5":
    best_model = DecisionTreeC45(train_data, train_target)
else:
    best_model = DecisionTreeCART(train_data, train_target)

# Evaluate the best model on the test set
test_predictions_c45 = DecisionTreeC45(train_data, train_target).predict(test_data)
test_predictions_cart = DecisionTreeCART(train_data, train_target).predict(test_data)

test_f1_c45 = f1_score(test_target, test_predictions_c45, pos_label="+")
test_f1_cart = f1_score(test_target, test_predictions_cart, pos_label="+")

# Print final test set results for both models
print("\nFinal Test Set Evaluation:")
print(f"C4.5 Test F1-Score: {test_f1_c45:.4f}")
print(f"CART Test F1-Score: {test_f1_cart:.4f}")


# Announce the final winner
final_winner = "C4.5" if test_f1_c45 > test_f1_cart else "CART"
print(f"\nFinal Best Model (Based on Test Set): {final_winner} ")

Best Model: C4.5 (C4.5 F1: 0.8127, CART F1: 0.2078)

Final Test Set Evaluation:
C4.5 Test F1-Score: 0.7742
CART Test F1-Score: 0.0308

Final Best Model (Based on Test Set): C4.5 
